<a href="https://colab.research.google.com/github/enigmatized/NoGambleNoFuture/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This originally started as seeing what if I took the same directional trade, if a move for a specific time frame made a 3sd move for that time frame.

An example would be: 9:02 has a 3 sd move, what I just took that trade.
What I found? Taking the opposite of that trade almost always leads to profits.
What is also intresting is that it is true on many time frames and on many assets.

This makes this very ideal for a trading bot.
Which is why it probably works, many people are doing this, only exagerating this dynamic.
Since, I am not the only one seeing this and taking this trade.
Which also means absolute effciency is essential.
The end goal is to make a c++ bot that is optimized for speed.
Running on an instance that is maximized for internet speed.


It is such a simple short term strategy, its stupid.


Anyway. I am far from that.

I am curious to try this out on.
It seems to backtest well on : EUR/USD, USD/JPY, CL and SPY.



What did you learn from this so far?


1.   Shorting tops is a hugely profitable trade if you can do it. The thing is if you can do it. Might be a worth a study in its own right.
2.   I have to now triple check the math, but the theory looks to hold. That taking the opposite side of a above 3sd move on any asset, is a good move. Pretty easy one to implement.


The next step is to optimize stop losses, optimial take profits and  maybe rolling into the trade when it goes against me.

----------------------------------------

The optimizing  values, especially stop loss values might be the trickest part.
Especially because since the way I am measuring success, is by closing periods. This would be easy to take into account smaller time frames as signals for exiting out of the trade.
I can imagine if I keep this part by period it might be too late.
I should consider the exchange I am using at the moment does not really allow having limit orders and market orders at the same time.


I think this optimization level might be excessive because the exchange limits you to how many calls you can make, that pining it can cause issues.

This is all thoughts, I need to test this to an extent.


---------------------------------------------

I also should have a freeze out. This is just a theory, but I imagine if there is a serious market trend down, and there are mulitple std moves, then it might not be a good idea to run this bot.




-------------------------------------------

How do I do this in c++?



Because this theoretically can work on multiple assets at the same time I need to take that into consideration when thinking about how t

1. c++

Asset ->


###optimizing your model in python
###It maybe some type of rule based
###Zscore for each time
1. Well have to build z-values for each currency I am trading
2. It may make sense to build the data sets up with optimal values in python
3. Then write those to JSON/CSV

-- Time, mean, std, zscore it must be larger than,
--

Figuring out stop loss will be an art in its own right.
I think that really should be done first before the build of the bot per say
Also maybe figuring out how to get the value and make an order right before a new candle or after.

------------------------------

C++ Bot

1. c++


2. Then have a timer, thread for each asset.
3. I think this will basically be the same?
4. I should check if there is a correlation effect with multiple assets before I pull the trigger on



In [ ]:
from collections import defaultdict
import requests
from time import sleep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import json
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from datetime import datetime, timedelta, time
from urllib.error import HTTPError
import statistics

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


CSVfiles = []
for file in os.listdir("./"):
  if file.endswith(".csv"):
    CSVfiles.append(file)
print(CSVfiles)


unixTimeConverter = lambda x : (datetime.fromtimestamp(x)).strftime('%Y-%m-%d %H:%M:%S')
unixTimeConverterNoDateJustTime = lambda x: (datetime.fromtimestamp(x) - timedelta(hours=4)).time()


allDfs = []


for x in CSVfiles:

  dfTemp = pd.read_csv(x)
  dfTemp['estTime']  = dfTemp['time'].apply(unixTimeConverter)
  dfTemp['timeOnly'] = dfTemp['time'].apply(unixTimeConverterNoDateJustTime)

  dfTemp['candleHeight']  = dfTemp.apply( lambda row:    max(row['open'],  row['close']) - min(row['open'],  row['close']), axis = 1 )
  allDfs.append(dfTemp)

df = allDfs[0]
#This is for generating if profit
for i in range(1, 15):
  df['P/L' +str(i) +'Short_']  = ( df['close'] - df['close'].shift(-i) )
  df['P/L' +str(i) +'Long_']  =  (  df['close'].shift(-i) - df['close']     )
  df['P/L' +str(i) +'Short']  = ( df['close'] - df['close'].shift(i) )
  df['P/L' +str(i) +'Long']  =  (  df['close'].shift(i) - df['close']     )

df = df.dropna()

['OANDA EURUSD, 60(2).csv']


In [ ]:
d_counts_average_vol            = defaultdict(list)
d_counts_average_highLow_diff   = defaultdict(list)

df['closeOpenDelta']   = (abs( df['close'] - df['open'] )).copy()
df['CloseOpenDelta_']  = (df['close'] - df['open']).copy()



for index, row in df.iterrows():
  #put volumes in d
  time_  = str(row['timeOnly'])
  volume = row['Volume']
  d_counts_average_vol[str(time_)].append(volume)

  # df[index, 'closeOpenDelta']   = abs( row['close'] - row['open'] )
  # df.loc[index, 'CloseOpenDelta_']  = df['close'] - row['open']

  #priceDiffInD
  price_delta = row['closeOpenDelta']
  d_counts_average_highLow_diff[str(time_)].append(price_delta)



<ipython-input-3-4b57aaa14623>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['closeOpenDelta']   = (abs( df['close'] - df['open'] )).copy()
<ipython-input-3-4b57aaa14623>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CloseOpenDelta_']  = (df['close'] - df['open']).copy()


In [ ]:
import statistics

std_values_with_time_as_keys = {}

mean_values_with_time_as_keys = {}

for k , v in d_counts_average_highLow_diff.items():

  # Calculate mean and standard deviation of the sample
  mean = statistics.mean(v)
  std_dev = statistics.stdev(v)
  std_values_with_time_as_keys[k] = std_dev
  mean_values_with_time_as_keys[k] = mean

df['z-score']= (df['close']).copy()

for index, row in df.iterrows():
  #put volumes in d
  time_  = str(row['timeOnly'])
  mean_ = mean_values_with_time_as_keys[time_]
  std_ = std_values_with_time_as_keys[time_]
  #wow so baically shorting highs is a very profitbale strategy, who knew.
  #WHo can call the top type of thing.
  #But it does seem rather self, fullfilling, but this is very backwards data, you already know the answer.
  #Damn I feel stupid
  df.loc[index, 'z-score'] = (row['closeOpenDelta'] - mean)/ std_
  # row['z-score'] = (row['closeOpenDelta'] - mean)/ std_

<ipython-input-4-b829f906f688>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['z-score']= (df['close']).copy()


In [ ]:
totalProfShort = defaultdict(float)
totalProfLong  = defaultdict(float)
totalPro       = defaultdict(float)

totalProfLongList = defaultdict(list)

# allBigSDMoves
for index, row in df.iterrows():
  if row['z-score'] >= 4:
    if row['CloseOpenDelta_'] > 0:
      for i in range(1,15):
        lookup = 'P/L' + str(i) +'Short'
        totalProfLong[lookup] +=row[lookup]
        totalProfLongList[lookup].append(row[lookup])
        print(row['CloseOpenDelta_'])

for k, v in totalProfLong.items():
  print(k, v)


for k, v in totalProfLongList.items():
  print(k, v)
print(len(totalProfLongList))

0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.0027599999999998737
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.006809999999999983
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.003530000000000033
0.006850000000000023
0.006850000000000023
0.006850000000000023
0.006850000000000023
0.006850000000000023

In [ ]:
index

21917

In [ ]:
df[df['z-score'] > 4.4]['z-score'].describe()

count    136.000000
mean       6.292723
std        2.182943
min        4.402684
25%        4.894591
50%        5.632471
75%        6.801954
max       18.694264
Name: z-score, dtype: float64

In [ ]:
df['z-score'].describe()

count    12348.000000
mean         0.038106
std          1.015914
min         -1.441537
25%         -0.622101
50%         -0.220376
75%          0.416632
max          9.933889
Name: z-score, dtype: float64

So I would like to visualize the spots that show this happening, just to see if I can spot a pattern that might be helpful to deal with. Rather than blindly coidng something.


In [ ]:
from datetime import datetime, timedelta, time
from   collections import defaultdict
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from   inspect import currentframe, getframeinfo
import requests
from   time import sleep
import os
import sys
import re

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def plotPandasDFWithCanldes(dataFrame_):

   fig = go.Figure(data=[go.Candlestick(x=dataFrame_['timeOnly'],

                open=dataFrame_['open'],

                high=dataFrame_['high'],

                low=dataFrame_['low'],

                close=dataFrame_['close'])])
   fig.update_xaxes(title_text='Date')

   fig.update_yaxes(title_text='Prices')

   fig.update_layout(xaxis_rangeslider_visible=True, # Set Range Slider Bar

                 title = 'Not bad') # Set Title

   fig.show()

In [ ]:
z_score_ = 4.4
print(":where is this going")

indexThatMatter = defaultdict(int)

for index, row in df.iterrows():
  if z_score_ < row['z-score']:
    indexThatMatter[index]=1

print(len(indexThatMatter))

count = 0
for index, row in df.iterrows():
  if indexThatMatter[index] == 1 and row['close'] - row['open'] > 0:
    print(index, "zscpre", row['z-score'])
    print("close", row['close'], "open", row['open'], "close-open", row['close'] - row['open'] , row['close'] - row['open'] > 0)

    min_df = df.iloc[[i for i in range(index, index+10)]]

    row1 = min_df.iloc[0]
    print("are rows, equal", row1.equals(row))


    plotPandasDFWithCanldes(min_df)
    count+=1
    if count == 30: break





:where is this going
136
1128 close 1.13517 open 1.12836 close-open 0.006809999999999983 True
are rows, equal False


1129 close 1.1387 open 1.13517 close-open 0.003530000000000033 True
are rows, equal False


1132 close 1.14284 open 1.13599 close-open 0.006850000000000023 True
are rows, equal False


1147 close 1.14708 open 1.14199 close-open 0.005090000000000039 True
are rows, equal False


1204 close 1.13194 open 1.12716 close-open 0.0047800000000000065 True
are rows, equal False


...

13044 close 1.13641 open 1.12916 close-open 0.007249999999999979 True
are rows, equal False


13449 close 1.123 open 1.11781 close-open 0.005190000000000028 True
are rows, equal False


13600 close 1.09424 open 1.08808 close-open 0.006160000000000165 True
are rows, equal False


13747 close 1.1045 open 1.10006 close-open 0.0044399999999999995 True
are rows, equal False


13953 close 1.10374 open 1.0985 close-open 0.005239999999999911 True
are rows, equal False


In [ ]:
z_score_ = 4.4
print(":where is this going")

start_index = df[df['z-score'] > 4.5].index.tolist()

count = 0
for index in start_index:
  if df.loc[index, 'close'] - df.loc[index, 'open'] > 0:
    #print(index, "zscpre", row['z-score'])
    #print("close", row['close'], "open", row['open'], "close-open", row['close'] - row['open'] , row['close'] - row['open'] > 0)

    min_df =  df.loc[index : index + 10] #df.iloc[[i for i in range(index, index+10)]]

    print(index)

    plotPandasDFWithCanldes(min_df)
    count+=1
    if count == 50: break





:where is this going
1128


1129


1132


1147


1204


...

17844


17910


17922


17929


18127


In [ ]:
start_index = df[df['z-score'] > 4.5].index.tolist()
start_index

[1030,
 1128,
 1129,
 1132,
 1147,
 1154,
 1160,
 1204,
 1217,
 1218,
 1219,
 1246,
 1248,
 1285,
 1323,
 1330,
 1338,
 1348,
 1351,
 1354,
 1356,
 1367,
 1369,
 1372,
 1378,
 1383,
 1401,
 1469,
 1940,
 2057,
 2123,
 2508,
 2653,
 2656,
 2765,
 2779,
 5256,
 5257,
 5259,
 5985,
 6360,
 7193,
 7290,
 7508,
 9068,
 11692,
 11918,
 13044,
 13274,
 13394,
 13437,
 13449,
 13535,
 13558,
 13600,
 13747,
 13953,
 14157,
 14245,
 14358,
 14406,
 14586,
 14603,
 15293,
 15439,
 15532,
 15631,
 15632,
 15782,
 15871,
 15898,
 15924,
 15943,
 16026,
 16041,
 16188,
 16260,
 16477,
 16502,
 16672,
 16677,
 16797,
 16806,
 16836,
 16974,
 16987,
 17040,
 17041,
 17165,
 17364,
 17518,
 17557,
 17678,
 17707,
 17757,
 17767,
 17844,
 17910,
 17921,
 17922,
 17929,
 18127,
 18146,
 18182,
 18185,
 18186,
 18228,
 18231,
 18396,
 18495,
 18726,
 18805,
 19241,
 19283,
 19335,
 19336,
 19813,
 19892,
 19897,
 19952,
 19953,
 20081,
 20120,
 20819,
 20933]